In [1]:
#Necessary Packages
import time
import requests
import numpy as np
import pandas as pd 
from bs4 import BeautifulSoup

In [2]:
def collect_data(course_urls_file):
    url_file = open(course_urls_file)
    urls = url_file.readlines()
    urls_filtered = [item[:-1] for item in urls]
    # print(urls_filtered)
    
    Title = []
    Most_view_in4_1 = []
    Most_view_in4_2 = []
    Rank_in4_1 = []
    Rank_in4_2 = []
    Rank_in4_3 = []
    Video_id = []

    for url in urls_filtered:
        html_content = requests.get(url)
        soup = BeautifulSoup(html_content.content, "lxml")
    
        video_title = soup.title.text.split(" – ")[0].split("YouTube Stats of ")[1]
        Title.append(video_title)
        
        most_view_in4 = soup.text.split("Most views in a day: ")[1].split("\n")[0].split(" ")
        most_view_in4[0] = int(most_view_in4[0].replace(',', ''))
        most_view_in4[1] = most_view_in4[1][1:-1]
        Most_view_in4_1.append(most_view_in4[0])
        Most_view_in4_2.append(most_view_in4[1])
        
        rank_in4 = soup.text.split("Peaked at #")
        if(len(rank_in4) == 1): 
            rank_in4 = ['nan', 'nan', 'nan']
        elif(len(rank_in4[1].split("\n")[0].split(" ")) <= 6):
            rank_in4 = [rank_in4[1].split("\n")[0].split(" ")[0], 'nan', rank_in4[1].split("\n")[0].split(" ")[4]]
        else: 
            rank_in4 = [rank_in4[1].split("\n")[0].split(" ")[0], rank_in4[1].split("\n")[0].split(" ")[2], rank_in4[1].split("\n")[0].split(" ")[7]]

        rank_in4[0] = float(rank_in4[0])
        rank_in4[1] = float(rank_in4[1])
        rank_in4[2] = float(rank_in4[2])
        
        Rank_in4_1.append(rank_in4[0])
        Rank_in4_2.append(rank_in4[1])
        Rank_in4_3.append(rank_in4[2])
        
        video_id = url[32:-5]
        Video_id.append(video_id)
    
    data = pd.DataFrame({"Title": Title,
                         "Most view per day": Most_view_in4_1,
                         "Most-view-date": Most_view_in4_2, 
                         "Highest rank": Rank_in4_1,
                         "Time to Highest rank": Rank_in4_2,
                         "Charted-duration": Rank_in4_3, 
                         "URL id:": Video_id})
    # print(data)
    return data

In [4]:
data_pokemon = collect_data("../data/kworb_video_url.txt") 
data_pokemon.to_csv('raw_kworb_data.txt.csv', index = False)